In [1]:
import numpy as np


class Topsis():

  TYPE_COST = 'cost'
  TYPE_BENEFIT = 'benefit'

  def __init__(self, alternatif, atribut):
    """
    Inisialisasi TOPSIS

    PARAMATER
    ---------

    alternatif: dictionary
      {
        nama_alternatif: list value
      }
    atribut: array
      array dictionary dengan value bobot dan jenis
      {
        value: int,
        type: TYPE_COST/ TYPE_BENEFIT
      }
    """
    self.alternatif_name = np.array(list(alternatif.keys()))
    self.alternatif_value = np.array(list(alternatif.values()))
    self.atribut = atribut

  def alternatif_ternormalisasi(self):
    """
    mendapatkan matriks alternatif ternormalisasi
    """
    alt_normal = np.zeros_like(self.alternatif_value)
    _, column = self.alternatif_value.shape
    
    for criteria_index in range(column):
      pow = np.power(self.alternatif_value[:, criteria_index], 2)
      sum = np.sum(pow)
      sqrt = np.sqrt(sum)
      alt_normal[:, criteria_index] = self.alternatif_value[:, criteria_index] / sqrt
    
    return alt_normal

  def bobot_ternormalisasi(self, alternatif, weight):
    """
    mendapatkan bobot ternormalisasi

    PARAMETER
    ---------

    alternatif: ndarray
      alternatif yang sudah dilakukan normalisasi
    
    weight: array
      bobot kriteria
    """
    normal = alternatif * weight
    return normal

  
  def solution(self, alternatif, type_weights):
    """
    mendapatkan solusi ideal positif dan solusi ideal negatif
    """
    ideal_positif = []
    ideal_negatif = []
    _, column = alternatif.shape

    for criteria_index, type_weight in zip(range(column), type_weights):
      maximum = max(alternatif[:, criteria_index])
      minimum = min(alternatif[:, criteria_index])
      if type_weight == self.TYPE_BENEFIT:
        ideal_positif.append(maximum)
        ideal_negatif.append(minimum)
      elif type_weight == self.TYPE_COST:
        ideal_positif.append(minimum)
        ideal_negatif.append(maximum)

    ideal_positif = np.array(ideal_positif)
    ideal_negatif = np.array(ideal_negatif)

    return ideal_positif, ideal_negatif


  def distance_solution(self, alternatif, ideal):
    """
    menghitung jarak ideal positif dan ideal negatif
    """
    row, _ = alternatif.shape
    distance = []

    for alternatif_index in range(row):
      minus = alternatif[alternatif_index] - ideal
      power = np.power(minus, 2)
      sum = np.sum(power)
      sqrt = np.sqrt(sum)
      distance.append(sqrt)

    distance = np.array(distance)

    return distance

  
  def value_preferensi(self, distance_positif, distance_negatif):
    """
    menghitung distance positif dan distance negatif
    """
    prefs = []
    for dist_positif, dist_negatif in zip(distance_positif, 
                                          distance_negatif):
      pref = dist_negatif / (dist_negatif + dist_positif)
      prefs.append(pref)

    prefs = np.array(prefs)

    return prefs 


  def preferensi_rank(self, preferensi):
    """
    mendapatkan rank
    """
    temp = preferensi.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(preferensi))
    data = {
        'alternatif': self.alternatif_name,
        'preferensi': preferensi,
        'rank': np.absolute(ranks - len(preferensi))
    }
    
    return data
      


  def fit(self):
    """
    menghitung hasil TOPSIS
    """
    alternatif_normal = self.alternatif_ternormalisasi()
    weights = self.atribut['weights']
    weights = np.array(weights)
        
    bobot_normal = self.bobot_ternormalisasi(alternatif_normal, weights)

    type_weights = self.atribut['type']

    ideal_positif, ideal_negatif = self.solution(bobot_normal, type_weights)

    distance_positif = self.distance_solution(bobot_normal, ideal_positif)
    distance_negatif = self.distance_solution(bobot_normal, ideal_negatif)

    preferensi = self.value_preferensi(distance_positif, distance_negatif)
    

    data = self.preferensi_rank(preferensi)
    return data

In [2]:
# Studi Kasus
import pandas as pd

alternatif = {
    "Kos Melati": [1.3, 0.3, 16, 4, 600],
    "Kos Mawar": [1.5, 0.2, 12, 3, 500],
    "Kos Sehat": [1.7, 0.1, 10, 2, 350],
    "Kos Asih": [2, 0.5, 18, 4, 750],
    "Kos Anggrek": [1.3, 0.2, 9, 5, 450]
}

atribut = {
    'weights': [4, 3, 5, 3, 4],
    'type': [Topsis.TYPE_COST,
             Topsis.TYPE_COST,
             Topsis.TYPE_BENEFIT,
             Topsis.TYPE_BENEFIT,
             Topsis.TYPE_COST]
}

topsis = Topsis(alternatif, atribut)
output = topsis.fit()

print('Alternatif')
print(pd.DataFrame.from_dict(alternatif))

print('\n\nAtribut')
print(pd.DataFrame.from_dict(atribut))

print('\n\nHasil TOPSIS')
print(pd.DataFrame.from_dict(output))

Alternatif
   Kos Melati  Kos Mawar  Kos Sehat  Kos Asih  Kos Anggrek
0         1.3        1.5        1.7       2.0          1.3
1         0.3        0.2        0.1       0.5          0.2
2        16.0       12.0       10.0      18.0          9.0
3         4.0        3.0        2.0       4.0          5.0
4       600.0      500.0      350.0     750.0        450.0


Atribut
   weights     type
0        4     cost
1        3     cost
2        5  benefit
3        3  benefit
4        4     cost


Hasil TOPSIS
    alternatif  preferensi  rank
0   Kos Melati    0.588656     1
1    Kos Mawar    0.559734     4
2    Kos Sehat    0.563155     3
3     Kos Asih    0.407594     5
4  Kos Anggrek    0.573797     2
